In [4]:
library(RPostgreSQL)
library(ggplot2)

Loading required package: DBI


In [5]:
m <- read.csv("../../preprocessing/municipalities_and_states/municipalities_preprocessed.csv",
             colClasses="character")

In [6]:
hdi <- read.csv("../../preprocessing/human_development_index/hdi_preprocessed.csv",
                colClasses=c("cve"="character",
                            "cve_ent"="character",
                            "cve_mun"="character"))

In [7]:
head(m); head(hdi)

,cve_ent,cve_mun,nom_mun,cve
1,17,001,amacuzac,17001
2,17,002,atlatlahucan,17002
3,17,003,axochiapan,17003
4,17,004,ayala,17004
5,17,005,coatlan del rio,17005
6,17,006,cuautla,17006


,cve,cve_ent,cve_mun,nom_ent,nom_mun,avg_years_schooling,expected_years_schooling,income_per_capita,infant_mortality_rate,education_index,income_index,healthcare_index,hdi_index
1,01001,01,001,aguascalientes,aguascalientes,9.553,12.87,17848.29,10.296,0.738,0.743,0.897,0.789
2,01002,01,002,aguascalientes,asientos,6.23,11.136,6877.029,13.211,0.555,0.606,0.861,0.662
3,01003,01,003,aguascalientes,calvillo,5.957,10.893,8764.943,12.229,0.536,0.641,0.873,0.67
4,01004,01,004,aguascalientes,cosio,7.007,11.851,9123.24,12.184,0.607,0.646,0.874,0.7
5,01005,01,005,aguascalientes,jesus maria,8.155,11.533,12156.48,13.972,0.646,0.688,0.852,0.723
6,01006,01,006,aguascalientes,pabellon de arteaga,8.197,12.22,12556.7,11.963,0.666,0.692,0.877,0.74


In [8]:
index <- hdi#[,c("cve", "hdi_index")]
head(index)

,cve,cve_ent,cve_mun,nom_ent,nom_mun,avg_years_schooling,expected_years_schooling,income_per_capita,infant_mortality_rate,education_index,income_index,healthcare_index,hdi_index
1,01001,01,001,aguascalientes,aguascalientes,9.553,12.87,17848.29,10.296,0.738,0.743,0.897,0.789
2,01002,01,002,aguascalientes,asientos,6.23,11.136,6877.029,13.211,0.555,0.606,0.861,0.662
3,01003,01,003,aguascalientes,calvillo,5.957,10.893,8764.943,12.229,0.536,0.641,0.873,0.67
4,01004,01,004,aguascalientes,cosio,7.007,11.851,9123.24,12.184,0.607,0.646,0.874,0.7
5,01005,01,005,aguascalientes,jesus maria,8.155,11.533,12156.48,13.972,0.646,0.688,0.852,0.723
6,01006,01,006,aguascalientes,pabellon de arteaga,8.197,12.22,12556.7,11.963,0.666,0.692,0.877,0.74


In [9]:
m <- merge(m, index)

In [10]:
head(m)

,cve_ent,cve_mun,nom_mun,cve,nom_ent,avg_years_schooling,expected_years_schooling,income_per_capita,infant_mortality_rate,education_index,income_index,healthcare_index,hdi_index
1,01,001,aguascalientes,01001,aguascalientes,9.553,12.87,17848.29,10.296,0.738,0.743,0.897,0.789
2,01,002,asientos,01002,aguascalientes,6.23,11.136,6877.029,13.211,0.555,0.606,0.861,0.662
3,01,003,calvillo,01003,aguascalientes,5.957,10.893,8764.943,12.229,0.536,0.641,0.873,0.67
4,01,004,cosio,01004,aguascalientes,7.007,11.851,9123.24,12.184,0.607,0.646,0.874,0.7
5,01,005,jesus maria,01005,aguascalientes,8.155,11.533,12156.48,13.972,0.646,0.688,0.852,0.723
6,01,006,pabellon de arteaga,01006,aguascalientes,8.197,12.22,12556.7,11.963,0.666,0.692,0.877,0.74


In [11]:
#https://en.wikipedia.org/?title=List_of_countries_by_Human_Development_Index
classify_hdi <- function(index) {
 if (index >= 0.8) "very high"
 else if (index >= 0.7) "high"
 else if (index >= 0.55) "medium"
 else "low"
}

In [12]:
m$hdi_class <- unlist(lapply(m$hdi_index, classify_hdi))

In [13]:
zones <- read.csv("states_geo_zones.csv", colClasses="character", skip=1)

In [14]:
m <- merge(m, zones)
head(m)

,cve_ent,nom_ent,cve_mun,nom_mun,cve,avg_years_schooling,expected_years_schooling,income_per_capita,infant_mortality_rate,education_index,income_index,healthcare_index,hdi_index,hdi_class,geo_zone
1,01,aguascalientes,009,tepezala,01009,6.463,11.736,8264.155,14.95,0.58,0.632,0.84,0.675,medium,north-central
2,01,aguascalientes,005,jesus maria,01005,8.155,11.533,12156.48,13.972,0.646,0.688,0.852,0.723,high,north-central
3,01,aguascalientes,008,san jose de gracia,01008,7.225,11.9,9257.589,15.329,0.617,0.649,0.835,0.694,medium,north-central
4,01,aguascalientes,003,calvillo,01003,5.957,10.893,8764.943,12.229,0.536,0.641,0.873,0.67,medium,north-central
5,01,aguascalientes,010,el llano,01010,5.975,10.68,7581.742,12.124,0.532,0.62,0.875,0.661,medium,north-central
6,01,aguascalientes,011,san francisco de los romo,01011,7.954,11.32,11283.69,13.104,0.632,0.677,0.863,0.717,high,north-central


In [15]:
write.csv(m, "municipalities_features.csv", row.names=FALSE)